In [1]:
import pandas as pd
import json

# Read the file line by line and parse each JSON object
people_txt_path = "/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc-snapshot-2025-10-14_1of30 (1)/psc-snapshot-2025-10-14_1of30.txt"

# Option 1: Read all lines into a list of dicts
with open(people_txt_path, 'r') as f:
    data = [json.loads(line) for line in f]

# Create DataFrame from the list of dicts
people_df = pd.DataFrame(data)
print(people_df.columns.tolist())
people_df.head()

['company_number', 'data']


,company_number,data
0,07434180,{'address': {'address_line_1': 'Boddington Lan...
1,08810260,"{'address': {'address_line_1': 'Godmans Lane',..."
2,06022970,{'address': {'address_line_1': '20 Rugby Road ...
3,08301480,"{'address': {'address_line_1': 'Station Road',..."
4,07428111,"{'address': {'address_line_1': 'Cottage', 'add..."


In [2]:
df2 = {"company_number":"04333110","data":{"address":{"address_line_1":"Gedney Marsh","address_line_2":"Holbeach","locality":"Spalding","postal_code":"PE12 9PB","premises":"Norfolk House Farm","region":"Lincolnshire"},"country_of_residence":"England","date_of_birth":{"month":2,"year":1971},"etag":"401ca7e16bf5effef24fd2abf9c0da723b84a74f","kind":"individual-person-with-significant-control","links":{"self":"/company/04333110/persons-with-significant-control/individual/KTvVTJijcv8RGXnU3Cub8GBf0Gc"},"name":"Mr Stuart William Piccaver","name_elements":{"forename":"Stuart","middle_name":"William","surname":"Piccaver","title":"Mr"},"nationality":"British","natures_of_control":["ownership-of-shares-25-to-50-percent","voting-rights-25-to-50-percent"],"notified_on":"2016-04-06"}} 

In [3]:
import json
import pandas as pd

psc_path = "/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc-snapshot-2025-10-14_1of30 (1)/psc-snapshot-2025-10-14_1of30.txt"

try:
    # NDJSON fast path
    chunk = next(pd.read_json(psc_path, lines=True, chunksize=1000))
    flat = pd.json_normalize(chunk.to_dict(orient='records'), sep='.')
except ValueError:
    # Fallback: TSV/CSV with a JSON 'data' column
    tsv = pd.read_csv(psc_path, sep="\t", nrows=10000)
    if 'data' in tsv.columns:
        parsed = []
        for x in tsv['data']:
            if isinstance(x, str):
                try:
                    parsed.append(json.loads(x))
                except Exception:
                    parsed.append({})
            elif isinstance(x, dict):
                parsed.append(x)
            else:
                parsed.append({})
        flat = pd.json_normalize(parsed, sep='.')
        if 'company_number' in tsv.columns and 'company_number' not in flat.columns:
            flat['company_number'] = tsv['company_number'].values[:len(flat)]
    else:
        flat = tsv  # best-effort

display(flat.head())
print(f"Rows: {len(flat)}, Columns: {len(flat.columns)}")

,company_number,data.address.address_line_1,data.address.address_line_2,data.address.country,data.address.locality,data.address.postal_code,data.address.premises,data.etag,data.identification.country_registered,data.identification.legal_authority,...,data.date_of_birth.year,data.name_elements.forename,data.name_elements.middle_name,data.name_elements.surname,data.name_elements.title,data.nationality,data.address.region,data.ceased_on,data.address.care_of,data.address.po_box
0,07434180,Boddington Lane,Boddington,England,Cheltenham,GL51 0TJ,The Manor,706b6e8773700f85361929bb5cd0214d630de6e0,England & Wales,Companies Act,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,08810260,Godmans Lane,Kirk Ella,NaN,Hull,HU10 7NY,Alvenga,224fe8afa8a6ff381ea0069dac5c6d4c995f15b2,NaN,NaN,...,1951.0,Michael,Charles,Saunders,Mr,British,NaN,NaN,NaN,NaN
2,06022970,20 Rugby Road Milverton,Leamington Spa,NaN,Warwickshire,CV32 6DG,NaN,1840e605a8948c04246e8d9904da789e2fdc4a7e,NaN,NaN,...,1945.0,Jean,Carol,Randle,Mrs,British,NaN,NaN,NaN,NaN
3,08301480,Station Road,NaN,England,Upminster,RM14 2SJ,"Suite 17, Essex House",0705b53fe3b34355fe183ae68013e8e1e19fda42,NaN,NaN,...,1983.0,Robert,NaN,Power,Mr,British,Essex,NaN,NaN,NaN
4,07428111,Cottage,Ardens Grafton,NaN,Alcester,B49 6DS,Malthouse,3ea7a8ff70ce98944c276648a875257e9b127428,NaN,NaN,...,1964.0,Andrew,John,Cain,Mr,British,Warwickshire,NaN,NaN,NaN


Rows: 1000, Columns: 30


In [4]:
out_path = "/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc_flat_1000.csv"
flat.to_csv(out_path, index=False)
print(f"Saved {len(flat)} rows x {len(flat.columns)} cols to {out_path}")

Saved 1000 rows x 30 cols to /home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc_flat_1000.csv


In [5]:

import requests
import csv
import time
import json
from datetime import datetime

# Configuration
API_KEY = "b8ca8790-2ffe-453e-a4e5-8e45b935e19a"  # Replace with your actual API key
BASE_URL = "https://api.company-information.service.gov.uk/advanced-search/companies"
OUTPUT_FILE = f"active_companies_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
BATCH_SIZE = 5000
RATE_LIMIT_DELAY = 2  # seconds between requests

# TEST MODE: Set to True to only fetch first few batches for testing
TEST_MODE = True
TEST_BATCHES = 3  # Number of batches to fetch in test mode

def fetch_companies(start_index, size):
    """Fetch a batch of companies from the API"""
    params = {
        'company_status': 'active',
        'size': size,
        'start_index': start_index
    }
    
    try:
        response = requests.get(
            BASE_URL,
            params=params,
            auth=(API_KEY, ''),  # API key as username, empty password
            timeout=30
        )
        
        # Detailed error reporting
        if response.status_code == 401:
            print("\n⚠️  AUTHENTICATION ERROR:")
            print("   - Check your API key is correct")
            print("   - Make sure it's a REST API key (not Streaming API)")
            print("   - Verify the key is registered at: https://developer.company-information.service.gov.uk/")
            print(f"   - Response: {response.text}")
            return None
        
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        if hasattr(e, 'response') and hasattr(e.response, 'text'):
            print(f"Response body: {e.response.text}")
        return None

def flatten_dict(d, parent_key='', sep='_'):
    """Flatten nested dictionaries for CSV compatibility"""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            # Convert lists to JSON strings
            items.append((new_key, json.dumps(v)))
        else:
            items.append((new_key, v))
    return dict(items)

def write_to_csv(companies, filename, write_header=False, existing_fieldnames=None):
    """Append companies to CSV file"""
    if not companies:
        return existing_fieldnames
    
    # Flatten all companies
    flattened_companies = [flatten_dict(company) for company in companies]
    
    # Collect all fieldnames from this batch
    batch_fieldnames = set()
    for company in flattened_companies:
        batch_fieldnames.update(company.keys())
    
    # Merge with existing fieldnames if this isn't the first batch
    if existing_fieldnames is not None:
        all_fieldnames = existing_fieldnames.union(batch_fieldnames)
    else:
        all_fieldnames = batch_fieldnames
    
    # Sort for consistency
    fieldnames = sorted(all_fieldnames)
    
    mode = 'w' if write_header else 'a'
    
    with open(filename, mode, newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, extrasaction='ignore', restval='')
        
        if write_header:
            writer.writeheader()
        
        writer.writerows(flattened_companies)
    
    return all_fieldnames

def main():
    print("Companies House Active Companies Scraper")
    print("=" * 50)
    
    if TEST_MODE:
        print("*** TEST MODE ENABLED ***")
        print(f"Will only fetch {TEST_BATCHES} batches ({TEST_BATCHES * BATCH_SIZE:,} records)")
        print("Set TEST_MODE = False to run full scrape")
        print()
    
    print(f"Output file: {OUTPUT_FILE}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Rate limit delay: {RATE_LIMIT_DELAY}s")
    print()
    
    start_index = 0
    total_retrieved = 0
    batch_number = 1
    write_header = True
    fieldnames_tracker = None
    
    while True:
        # Check if we've hit test mode limit
        if TEST_MODE and batch_number > TEST_BATCHES:
            print()
            print("=" * 50)
            print(f"TEST MODE COMPLETE: {total_retrieved:,} companies saved")
            print("Check the CSV file to verify the data looks correct.")
            print("Set TEST_MODE = False to run the full scrape.")
            break
        
        print(f"Fetching batch {batch_number} (starting at index {start_index})...")
        
        data = fetch_companies(start_index, BATCH_SIZE)
        
        if data is None:
            print("Failed to fetch data. Stopping.")
            break
        
        companies = data.get('items', [])
        total_available = data.get('total_results', 0)
        
        if not companies:
            print("No more companies to retrieve.")
            break
        
        # Write to CSV and track fieldnames
        fieldnames_tracker = write_to_csv(companies, OUTPUT_FILE, write_header=write_header, existing_fieldnames=fieldnames_tracker)
        write_header = False  # Only write header once
        
        total_retrieved += len(companies)
        print(f"✓ Retrieved {len(companies)} companies")
        
        
        
        # Check if we've retrieved everything
        if total_retrieved >= total_available:
            print("All companies retrieved!")
            break
        
        # Move to next batch
        start_index += BATCH_SIZE
        batch_number += 1
        
        # Rate limiting
        print(f"Waiting {RATE_LIMIT_DELAY}s before next request...")
        time.sleep(RATE_LIMIT_DELAY)
    
    print()
    print("=" * 50)
    print(f"COMPLETE: {total_retrieved:,} companies saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()


Companies House Active Companies Scraper
*** TEST MODE ENABLED ***
Will only fetch 3 batches (15,000 records)
Set TEST_MODE = False to run full scrape

Output file: active_companies_20251024_143841.csv
Batch size: 5000
Rate limit delay: 2s

Fetching batch 1 (starting at index 0)...
✓ Retrieved 5000 companies
All companies retrieved!

COMPLETE: 5,000 companies saved to active_companies_20251024_143841.csv


In [ ]:
import os, requests
BASE = "https://api.company-information.service.gov.uk"  # or https://api-sandbox.company-information.service.gov.uk

session = requests.Session()
session.auth = (API_KEY, "")

def company_name_by_number(company_number: str) -> str | None:
    r = session.get(f"{BASE}/company/{company_number}", timeout=10)
    if r.status_code == 200:
        data = r.json()
        return data.get("company_name")
    return None

def search_company_name(query: str, limit: int = 1) -> list[str]:
    r = session.get(f"{BASE}/search/companies", params={"q": query, "items_per_page": limit}, timeout=10)
    if r.status_code == 200:
        data = r.json()
        names = []
        for item in data.get("items", []):
            # search returns 'title' (display name); fallback to 'company_name' if present
            names.append(item.get("title") or item.get("company_name"))
        return [n for n in names if n]
    return []

# Examples
test_company_number = flat['company_number'][3]
print(flat['company_number'][3])

test_company_number = "08810260"
print(company_name_by_number(test_company_number))
import time

08301480
WILLERFOSS HOMES LIMITED


In [ ]:
import time
import pandas as pd
import json

def fetch_full_company_profile(company_number: str, max_retries: int = 3) -> dict | None:
    """Fetch complete company profile from API with rate-limit handling"""
    url = f"{BASE}/company/{company_number}"
    
    for attempt in range(max_retries):
        try:
            r = session.get(url, timeout=10)
            
            if r.status_code == 200:
                return r.json()
            
            elif r.status_code == 429:
                retry_after = r.headers.get('Retry-After')
                if retry_after:
                    wait = int(retry_after)
                else:
                    wait = 2 ** (attempt + 1)
                
                print(f"  ⚠️  Rate limited. Waiting {wait}s (attempt {attempt + 1}/{max_retries})...")
                time.sleep(wait)
            
            elif r.status_code == 404:
                return None
            
            else:
                print(f"  ✗ HTTP {r.status_code}: {r.text[:100]}")
                return None
                
        except Exception as e:
            print(f"  ✗ Request error: {e}")
            if attempt < max_retries - 1:
                time.sleep(1 + attempt)
            
    return None

def enrich_flat_with_api_data(df, delay=0.6, max_retries=3):
    """
    Enrich existing rows in df by fetching API data for each company_number.
    Adds new columns from API response without adding new rows.
    """
    api_columns_initialized = False
    api_column_list = []
    
    print(f"Enriching {len(df)} rows with API data...")
    print(f"Base delay: {delay}s, Max retries: {max_retries}")
    print("-" * 60)
    
    for i, idx in enumerate(df.index, 1):
        company_num = str(df.loc[idx, 'company_number']).strip()
        
        print(f"[{i}/{len(df)}] Fetching {company_num}...", end=" ")
        
        profile = fetch_full_company_profile(company_num, max_retries=max_retries)
        
        if profile:
            # Flatten the API response
            flat_profile = pd.json_normalize(profile, sep='.')
            
            # First successful fetch: initialize columns
            if not api_columns_initialized:
                api_column_list = list(flat_profile.columns)
                for col in api_column_list:
                    if col not in df.columns:
                        df[col] = None
                api_columns_initialized = True
                print(f"✓ (initialized {len(api_column_list)} API columns)")
            else:
                print("✓")
            
            # Update this row with API data - FIXED: handle lists/dicts properly
            for col in api_column_list:
                if col in flat_profile.columns:
                    val = flat_profile[col].iloc[0]
                    
                    # Convert lists and dicts to JSON strings for safe storage
                    if isinstance(val, (list, dict)):
                        df.at[idx, col] = json.dumps(val)
                    else:
                        df.at[idx, col] = val
        else:
            print("✗ No data")
        
        # Rate limiting
        if i < len(df):
            time.sleep(delay)
    
    print("-" * 60)
    print(f"✓ Complete! Final shape: {df.shape}")
    return df

# Usage
flat = enrich_flat_with_api_data(flat, delay=0.6, max_retries=3)

# Save enriched data
flat.to_csv("/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc_flat_500_enriched.csv", index=False)
print("Saved to psc_flat_500_enriched.csv")

Enriching 1000 rows with API data...
Base delay: 0.6s, Max retries: 3
------------------------------------------------------------
[1/1000] Fetching 07434180... ✓ (initialized 45 API columns)
[2/1000] Fetching 08810260... ✓
[3/1000] Fetching 06022970... ✓
[4/1000] Fetching 08301480... ✓
[5/1000] Fetching 07428111... ✓
[6/1000] Fetching 07425821... ✓
[7/1000] Fetching 07874121... ✓
[8/1000] Fetching 10523951... ✓
[9/1000] Fetching 10518671... ✓
[10/1000] Fetching 04333110... ✓
[11/1000] Fetching 09325250... ✓
[12/1000] Fetching 04334851... ✓
[13/1000] Fetching 07882791... ✓
[14/1000] Fetching 07459990... ✓
[15/1000] Fetching 04995740... ✓
[16/1000] Fetching 07855501... ✓
[17/1000] Fetching 07467371... ✓
[18/1000] Fetching 07462271... ✓
[19/1000] Fetching 08272140... ✓
[20/1000] Fetching 10063540... ✓
[21/1000] Fetching 10527261... ✓
[22/1000] Fetching 00968810... ✓
[23/1000] Fetching 08811261... ✓
[24/1000] Fetching 04120121... ✓
[25/1000] Fetching SC335301... ✓
[26/1000] Fetching SC241

In [ ]:
flat.to_csv("/home/ae25872/codebase/proai/Common-Crawl---Autumn-2025/psc_flat_500_enriched.csv", index=False)

In [ ]:
import pandas as pd


# Example: fetch profile for test_company_number and add to flat
test_company_number = "07434180"
profile = fetch_full_company_profile(test_company_number)

if profile:
    # Flatten the nested dict to match flat's structure
    flat_profile = pd.json_normalize(profile, sep='.')
    
    # Append to flat (concat handles mismatched columns gracefully)
    flat = pd.concat([flat, flat_profile], ignore_index=True)
    
    print(f"Added company {test_company_number} to flat")
    print(f"New shape: {flat.shape}")
    display(flat.tail(3))  # show last 3 rows including the new one
else:
    print(f"Failed to fetch company {test_company_number}")

Added company 07434180 to flat
New shape: (501, 75)


,company_number,data.address.address_line_1,data.address.address_line_2,data.address.country,data.address.locality,data.address.postal_code,data.address.premises,data.etag,data.identification.country_registered,data.identification.legal_authority,data.identification.legal_form,data.identification.place_registered,data.identification.registration_number,data.kind,data.links.self,data.name,data.natures_of_control,data.notified_on,data.country_of_residence,data.date_of_birth.month,data.date_of_birth.year,data.name_elements.forename,data.name_elements.middle_name,data.name_elements.surname,data.name_elements.title,data.nationality,data.address.region,data.ceased_on,data.address.care_of,data.address.po_box,sic_code,can_file,company_name,company_status,date_of_creation,etag,has_been_liquidated,has_charges,has_insolvency_history,jurisdiction,last_full_members_list_date,previous_company_names,registered_office_is_in_dispute,sic_codes,type,undeliverable_registered_office_address,has_super_secure_pscs,accounts.accounting_reference_date.day,accounts.accounting_reference_date.month,accounts.last_accounts.made_up_to,accounts.last_accounts.period_end_on,accounts.last_accounts.period_start_on,accounts.last_accounts.type,accounts.next_accounts.due_on,accounts.next_accounts.overdue,accounts.next_accounts.period_end_on,accounts.next_accounts.period_start_on,accounts.next_due,accounts.next_made_up_to,accounts.overdue,confirmation_statement.last_made_up_to,confirmation_statement.next_due,confirmation_statement.next_made_up_to,confirmation_statement.overdue,links.persons_with_significant_control,links.self,links.charges,links.filing_history,links.officers,registered_office_address.address_line_1,registered_office_address.address_line_2,registered_office_address.country,registered_office_address.locality,registered_office_address.postal_code,registered_office_address.region
498,SC458310,Moorpark Avenue,NaN,Scotland,Airdrie,ML6 8EU,13,5e6f98187eb3b49f01ffcd116f1cc0d2e444e0a8,NaN,NaN,NaN,NaN,NaN,individual-person-with-significant-control,/company/SC458310/persons-with-significant-con...,Mr Robert Allan,"[ownership-of-shares-75-to-100-percent, voting...",2016-06-30,Scotland,8.0,1976.0,Robert,NaN,Allan,Mr,British,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,09896371,Colindeep Lane,NaN,United Kingdom,Colindale,NW9 6BX,5 Technology Park,b4767f84221e3f996d58fea1f4226baadcc60a7c,NaN,NaN,NaN,NaN,NaN,individual-person-with-significant-control,/company/09896371/persons-with-significant-con...,Mr Niraj Shah,"[ownership-of-shares-25-to-50-percent, voting-...",2016-04-06,England,3.0,1979.0,Niraj,NaN,Shah,Mr,British,London,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,07434180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,CASTLEGATE BUSINESS PARK LIMITED,active,2010-11-09,025aebda264b06be717a19aa5101f5bf5713b4b0,False,False,False,england-wales,2015-11-09,"[{'ceased_on': '2011-03-30', 'effective_from':...",False,[68209],ltd,False,False,31,03,2024-03-31,2024-03-31,2023-04-01,micro-entity,2025-12-31,False,2025-03-31,2024-04-01,2025-12-31,2025-03-31,False,2024-11-09,2025-11-23,2025-11-09,False,/company/07434180/persons-with-significant-con...,/company/07434180,/company/07434180/charges,/company/07434180/filing-history,/company/07434180/officers,The Manor Boddington Lane,Boddington,England,Cheltenham,GL51 0TJ,Gloucestershire


In [ ]:
def sic_codes_by_number(company_number: str) -> list[str] | None:
    r = session.get(f"{BASE}/company/{company_number}", timeout=10)
    if r.status_code == 200:
        data = r.json()
        sic = data.get("sic_codes")
        return sic if isinstance(sic, list) else None
    return None

def primary_sic_code(company_number: str) -> str | None:
    codes = sic_codes_by_number(company_number)
    return codes[0] if codes else None